In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
# 构造分页数字列表
page_indexs = range(0, 250, 25) #豆瓣top250的分页构造0→25→50直到250

In [18]:
list(page_indexs)

[0, 25, 50, 75, 100, 125, 150, 175, 200, 225]

In [19]:
def download_all_htmls():
    """
    下载所有列表页面的HTML，用于后续的分析
    """
    htmls = []
    for idx in page_indexs:
        url = f"https://movie.douban.com/top250?start={idx}&filter="  #讲豆瓣top250的分页参数替换成建构的list
        print("craw html:", url)
        #为了屏蔽豆瓣的反爬功能，增强headers
        r = requests.get(url,
                        headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"})
        if r.status_code != 200:
            raise Exception("error")
        htmls.append(r.text)
    return htmls

In [20]:
# 执行爬取
htmls = download_all_htmls()

craw html: https://movie.douban.com/top250?start=0&filter=
craw html: https://movie.douban.com/top250?start=25&filter=
craw html: https://movie.douban.com/top250?start=50&filter=
craw html: https://movie.douban.com/top250?start=75&filter=
craw html: https://movie.douban.com/top250?start=100&filter=
craw html: https://movie.douban.com/top250?start=125&filter=
craw html: https://movie.douban.com/top250?start=150&filter=
craw html: https://movie.douban.com/top250?start=175&filter=
craw html: https://movie.douban.com/top250?start=200&filter=
craw html: https://movie.douban.com/top250?start=225&filter=


In [21]:
htmls[0]

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-windows ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n豆瓣电影 Top 250\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">\n    \n    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">\n    <link href="https://img3.doubanio.com/f/shire/859dba5cddc7ed1435808cf5a8ddde5792cd6e0c/css/douban.css" rel="stylesheet" type="text/css">\n    <link href="https://img3.doubanio.com/f/shire/db02bd3a4c78de56425ddeedd748a6804af60ee9/css/separation/_all.css" rel="style

In [28]:
def parse_single_html(html):  #传入一个html并返回数据
    """
    解析单个HTML，得到数据
    @return list({"link", "title", [label]})
    """
    soup = BeautifulSoup(html, 'html.parser')
    article_items = (
        soup.find("div", class_="article")  #对照html找出我们信息所在的位置及内容
            .find("ol", class_="grid_view")
            .find_all("div", class_="item")
    )
    datas = []
    for article_item in article_items:
        rank = article_item.find("div", class_="pic").find("em").get_text()
        info = article_item.find("div", class_="info")
        title = info.find("div", class_="hd").find("span", class_="title").get_text()
        stars = (
            info.find("div", class_="bd")
                .find("div", class_="star")
                .find_all("span")   #共4个span，所以使用find_all
        )
        rating_star = stars[0]["class"][0]  #class属性有多个，取第【0】个
        rating_num = stars[1].get_text()
        comments = stars[3].get_text()  #评价人数
        
        datas.append({
            "rank":rank,
            "title":title,
            "rating_star":rating_star.replace("rating","").replace("-t",""),  #去掉评级的前缀和后缀
            "rating_num":rating_num,
            "comments":comments.replace("人评价", "")  #去掉后缀“人评价”
        })
    return datas
    

In [29]:
import pprint
pprint.pprint(parse_single_html(htmls[0]))

[{'comments': '2240495',
  'rank': '1',
  'rating_num': '9.7',
  'rating_star': '5',
  'title': '肖申克的救赎'},
 {'comments': '1663435',
  'rank': '2',
  'rating_num': '9.6',
  'rating_star': '5',
  'title': '霸王别姬'},
 {'comments': '1687462',
  'rank': '3',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '阿甘正传'},
 {'comments': '1872302',
  'rank': '4',
  'rating_num': '9.4',
  'rating_star': '45',
  'title': '这个杀手不太冷'},
 {'comments': '1646131',
  'rank': '5',
  'rating_num': '9.4',
  'rating_star': '45',
  'title': '泰坦尼克号'},
 {'comments': '1043219',
  'rank': '6',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '美丽人生'},
 {'comments': '1763057',
  'rank': '7',
  'rating_num': '9.4',
  'rating_star': '45',
  'title': '千与千寻'},
 {'comments': '861213',
  'rank': '8',
  'rating_num': '9.5',
  'rating_star': '5',
  'title': '辛德勒的名单'},
 {'comments': '1638668',
  'rank': '9',
  'rating_num': '9.3',
  'rating_star': '45',
  'title': '盗梦空间'},
 {'comments': '1120507',
  'rank': '10',
  '

In [31]:
#对页面进行再分析
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(html))

In [32]:
all_datas

[{'rank': '1',
  'title': '肖申克的救赎',
  'rating_star': '5',
  'rating_num': '9.7',
  'comments': '2240495'},
 {'rank': '2',
  'title': '霸王别姬',
  'rating_star': '5',
  'rating_num': '9.6',
  'comments': '1663435'},
 {'rank': '3',
  'title': '阿甘正传',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '1687462'},
 {'rank': '4',
  'title': '这个杀手不太冷',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1872302'},
 {'rank': '5',
  'title': '泰坦尼克号',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1646131'},
 {'rank': '6',
  'title': '美丽人生',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '1043219'},
 {'rank': '7',
  'title': '千与千寻',
  'rating_star': '45',
  'rating_num': '9.4',
  'comments': '1763057'},
 {'rank': '8',
  'title': '辛德勒的名单',
  'rating_star': '5',
  'rating_num': '9.5',
  'comments': '861213'},
 {'rank': '9',
  'title': '盗梦空间',
  'rating_star': '45',
  'rating_num': '9.3',
  'comments': '1638668'},
 {'rank': '10',
  'title': '忠犬八公的故事',
  'rat

In [33]:
len(all_datas)

250

In [35]:
#存入excel
import pandas as pd
df = pd.DataFrame(all_datas)
df

,rank,title,rating_star,rating_num,comments
0,1,肖申克的救赎,5,9.7,2240495
1,2,霸王别姬,5,9.6,1663435
2,3,阿甘正传,5,9.5,1687462
3,4,这个杀手不太冷,45,9.4,1872302
4,5,泰坦尼克号,45,9.4,1646131
...,...,...,...,...,...
245,246,小萝莉的猴神大叔,4,8.4,391622
246,247,追随,45,8.9,145379
247,248,崖上的波妞,45,8.5,353389
248,249,黑鹰坠落,45,8.7,231692


In [36]:
df.to_excel("豆瓣电影TOP250.xlsx")